# Keras / TensorFlow Playground

https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html

I used the code from the link above to get started.

In [18]:
import tensorflow as tf
import numpy as np

sess = tf.Session() #creates the tensorflow 'instance'

from keras import backend as K
K.set_session(sess) #associates the tensorflow instance created above with Keras

### After creating the tensorflow instance and pointing Keras to it, we build the network. First, we'll create an input and label 'placeholder' so we can proceed with the network before loading the image data ( I assume ).

In [19]:
# this placeholder will contain our input digits, as flat vectors
img = tf.placeholder(tf.float32, shape=(None, 784))
# img is used in the first layer below, labels is used one cell further
# img = array(img).reshape(
# this placeholder is for the labels
labels = tf.placeholder(tf.float32, shape=(None, 10))

In [22]:
from keras.layers import Dense

# Keras layers can be called on TensorFlow tensors:
x = Dense(128, activation='relu')(img) 
print x
print img
# fully-connected layer with 128 units and ReLU activation
x = Dense(128, activation='relu')(x)
print x
print img
preds = Dense(10, activation='softmax')(x) 
# output layer with 10 units and a softmax activation

Tensor("dense_16/Relu:0", shape=(?, 128), dtype=float32)
Tensor("Placeholder_4:0", shape=(?, 784), dtype=float32)
Tensor("dense_17/Relu:0", shape=(?, 128), dtype=float32)
Tensor("Placeholder_4:0", shape=(?, 784), dtype=float32)


In [13]:
# Defining the loss function

from keras.objectives import categorical_crossentropy
loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

# Training the model with a TensorFlow optimizer: There are some 'knobs' in this section: GradientDescentOptimizer value, and minimize(loss).
### Pay attention to the global_values_initializer() - it appears to be necessary after loading the dataset to tie it to the tensorflow instance. (*guess*)


This is where you would insert your own images to train a model for a different purpose,
ie object detection.

In [14]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

# Run training loop
with sess.as_default():
    for i in range(100):
        batch = mnist_data.train.next_batch(50)
        train_step.run(feed_dict={img: batch[0],
                                  labels: batch[1]})

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Now we can evaluate the model...

I really don't think this is correct: [ 1.  1.  1. ...,  1.  1.  1.]

Got this on the second run - [ 1.  1.  1. ...,  0.  1.  1.]

In [15]:
from keras.metrics import categorical_accuracy as accuracy

acc_value = accuracy(labels, preds)
with sess.as_default():
    print acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels})

[ 1.  1.  1. ...,  0.  1.  1.]


In this case, we use Keras only as a syntactical shortcut to generate an op that maps some tensor(s) input to some tensor(s) output, and that's it. The optimization is done via a native TensorFlow optimizer rather than a Keras optimizer. We don't even use any Keras Model at all!

A note on the relative performance of native TensorFlow optimizers and Keras optimizers: there are slight speed differences when optimizing a model "the Keras way" vs. with a TensorFlow optimizer. Somewhat counter-intuitively, Keras seems faster most of the time, by 5-10%. However these differences are small enough that it doesn't really matter, at the end of the day, whether you optimize your models via Keras optimizers or native TF optimizers.

##Different behaviors during training and testing##

Some Keras layers (e.g. Dropout, BatchNormalization) behave differently at training time and testing time. You can tell whether a layer uses the "learning phase" (train/test) by printing layer.uses_learning_phase, a boolean: True if the layer has a different behavior in training mode and test mode, False otherwise.

If your model includes such layers, then you need to specify the value of the learning phase as part of feed_dict, so that your model knows whether to apply dropout/etc or not.

The Keras learning phase (a scalar TensorFlow tensor) is accessible via the Keras backend:

In [16]:
from keras import backend as K
print K.learning_phase()

Tensor("keras_learning_phase:0", dtype=bool)


To make use of the learning phase, simply pass the value "1" (training mode) or "0" (test mode) to feed_dict:

In [17]:
# train mode
with sess.as_default():
    train_step.run(feed_dict={x: batch[0], labels: batch[1], K.learning_phase(): 1})

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float and shape [?,784]
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op u'Placeholder_2', defined at:
  File "/Users/brandonpippin/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/brandonpippin/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/brandonpippin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/brandonpippin/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/brandonpippin/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/brandonpip